<a href="https://colab.research.google.com/github/bariscelikW/nlptepe_dotsOCR/blob/main/nlptepe_dotsOCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/rednote-hilab/dots.ocr.git
%cd dots.ocr

Cloning into 'dots.ocr'...
remote: Enumerating objects: 204, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 204 (delta 42), reused 30 (delta 30), pack-reused 149 (from 2)
Receiving objects: 100% (204/204), 35.84 MiB | 21.92 MiB/s, done.
Resolving deltas: 100% (79/79), done.
/content/dots.ocr


In [2]:
!pip install torch==2.7.0 torchvision==0.22.0 torchaudio==2.7.0 --index-url https://download.pytorch.org/whl/cu128
!pip install -e .

Looking in indexes: https://download.pytorch.org/whl/cu128
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 GB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 105.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 609.6/609.6 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 123.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 726.9/726.9 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 65.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.4/260.4 MB 3.1 MB/s eta 0:

In [3]:
!python3 tools/download_model.py


Attention: The model save dir dots.ocr should be replace by a name without `.` like DotsOCR, util we merge our code to transformers.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:982: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(
Fetching 20 files:   0% 0/20 [00:00<?, ?it/s]
config.json: 1.47kB [00:00, 2.72MB/s]

configuration_dots.py: 2.93kB [00

In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoProcessor, AutoTokenizer
from qwen_vl_utils import process_vision_info
from dots_ocr.utils import dict_promptmode_to_prompt

model_path = "./weights/DotsOCR"
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    attn_implementation="flash_attention_2",
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True
)
processor = AutoProcessor.from_pretrained(model_path, trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [27]:


image_path = "/content/sinav2.png"
prompt = """Please output the layout information from the PDF image, including each layout element's bbox, its category, and the corresponding text content within the bbox.

1. Bbox format: [x1, y1, x2, y2]

2. Layout Categories: The possible categories are ['Caption', 'Footnote', 'Formula', 'List-item', 'Page-footer', 'Page-header', 'Picture', 'Section-header', 'Table', 'Text', 'Title'].

3. Text Extraction & Formatting Rules:
    - Picture: For the 'Picture' category, the text field should be omitted.
    - Formula: Format its text as LaTeX.
    - Table: Format its text as HTML.
    - All Others (Text, Title, etc.): Format their text as Markdown.

4. Constraints:
    - The output text must be the original text from the image, with no translation.
    - All layout elements must be sorted according to human reading order.

5. Final Output: The entire output must be a single JSON object.
"""

messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "image": image_path
                },
                {"type": "text", "text": prompt}
            ]
        }
    ]

# Preparation for inference
text = processor.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)

inputs = inputs.to("cuda")

# Inference: Generation of the output
generated_ids = model.generate(**inputs, max_new_tokens=24000)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


['[{"bbox": [10, 3, 377, 22], "category": "Section-header", "text": "Soru 1. (15 puan) Algoritm a Analizi (Analysis of Algorithms)"}, {"bbox": [10, 37, 568, 74], "category": "Text", "text": "Aşağıdaki verilen kod parçaları için çalışma sürelerinin büyüme düzenini (order of growth)\\nN\'nin bir fonksiyonu cinsinden --notasyonu ile belirtiniz."}, {"bbox": [10, 92, 90, 109], "category": "Section-header", "text": "(a) (8 puan)"}, {"bbox": [10, 127, 261, 194], "category": "Text", "text": "int m = 0;\\nfor (int i = N; i >0; i--)\\n    for (int j = 0; j < i; j++)\\n        m++;"}, {"bbox": [10, 194, 276, 257], "category": "Text", "text": "Büyüme Düzeni: ~ O(N²)"}, {"bbox": [10, 472, 90, 489], "category": "Section-header", "text": "(b) (7 puan)"}, {"bbox": [10, 506, 292, 574], "category": "Text", "text": "int k = 0;\\nfor (int i = 1; i <= N; i++)\\n    for (int j = 1; j <= N; j++)\\n        k++;"}, {"bbox": [10, 570, 363, 644], "category": "Text", "text": "Büyüme Düzeni: ~ O(n · log(n))"}]']


In [8]:
import json

# OCR'dan gelen çıktının bir liste içinde olduğunu varsayalım.
# Eğer doğrudan string geliyorsa output_text[0] yerine output_text kullanın.
try:
    # 1. JSON dizesini bir Python listesine/sözlüğüne dönüştür
    # Not: DotsOCR modeli bazen "```json" gibi işaretçilerle başlayabilir, bunları temizliyoruz.
    clean_json_string = output_text[0].replace("```json\n", "").replace("\n```", "")
    layout_data = json.loads(clean_json_string)

    # 2. Tüm metin içeriklerini okuma sırasına göre birleştir
    # Modelin çıktısı zaten okuma sırasına göre sıralanmış olmalı.
    ogrenci_cevabi_parcalari = []
    for element in layout_data:
        # 'Picture' gibi metin içermeyen kategorileri atla [cite: 33]
        if 'text' in element and element['text']:
            ogrenci_cevabi_parcalari.append(element['text'])

    # 3. Tüm metin parçalarını tek bir metin haline getir
    ogrenci_cevabi_tamami = "\n".join(ogrenci_cevabi_parcalari)

    print("--- OCR'DAN AYIKLANAN ÖĞRENCİ CEVABI ---")
    print(ogrenci_cevabi_tamami)
    print("-" * 40)

except json.JSONDecodeError as e:
    print(f"Hata: OCR çıktısı geçerli bir JSON formatında değil. Hata: {e}")
    print("Alınan Çıktı:", output_text[0])
    ogrenci_cevabi_tamami = "" # Hata durumunda metni boş bırak

--- OCR'DAN AYIKLANAN ÖĞRENCİ CEVABI ---
Questions
1) In $\mathbb{R}_2$, define $(x_1y) \oplus (x',y') = (x+x', y+y')$ and $c \odot (x,y) = (0,0)$. Is $\mathbb{R}_2$ a vector space with $\oplus$ and $\odot$ ? (c $\in \mathbb{R}$)
2) Let V be the set of all positive real numbers. Define $u \oplus v = u \cdot v$, $c \odot v = vc$ for any u, v $\in V$ and $c \in \mathbb{R}$. Prove that V is a vector space.
3) Determine whether or not the following set W is a subspace of V.
$$
a) W = \{ (2t, -3t) \mid t \in \mathbb{R} \}, V = \mathbb{R}_2 .
$$
$$
b) W = \{ (2t+3, -4t) \mid t \in \mathbb{R} \}, V = \mathbb{R}_2
$$
$$
c) W = \{ (a,b,c) | a,b,c \in \mathbb{R}, 3a+b-2c=0 \}, V = \mathbb{R}_3 .
$$
$$
d) W = \{ a_0 + a_1 t + a_2 t^2 \mid a_i \in \mathbb{R}, a_1 = 3a_0 \}, V = \mathbb{P}_2
$$
$$
e) W = \left\{ \begin{bmatrix} a & b & c \\ d & e & f \end{bmatrix} \mid a,b,c,d,e,f \in \mathbb{R}, a+c=0, b+d+f=0 \right\}, V = \mathbb{M}_{2,3}
$$
$$
f) V = C(-\infty, \infty) = \{f \mid f : \mathbb{R}

In [9]:
# Gerekli kütüphaneyi yükle
!pip install -U sentence-transformers

from sentence_transformers import SentenceTransformer, util

# Sadece öğrenci cevabı başarıyla ayıklandıysa devam et
if ogrenci_cevabi_tamami:
    # 1. Türkçe için ön-eğitilmiş bir anlamsal benzerlik modeli yükle
    model_semantic = SentenceTransformer('dbmdz/bert-base-turkish-cased')

    # 2. Akademisyenin sisteme girdiği cevap anahtarını tanımla
    # Bu kısmı kendi projenizde dinamik olarak almalısınız.
    cevap_anahtari = """
    Prokaryot hücreler, zarla çevrili bir çekirdek ve organellere sahip değildir.
    Genetik materyal sitoplazmada serbest halde bulunur ve ribozom dışında belirgin bir organeli yoktur.
    Bu hücre tipi genellikle bakteriler ve arkelerde görülür.
    """

    # 3. Öğrenci cevabını ve cevap anahtarını vektörlere dönüştür (embedding)
    embedding_ogrenci = model_semantic.encode(ogrenci_cevabi_tamami, convert_to_tensor=True)
    embedding_anahtar = model_semantic.encode(cevap_anahtari, convert_to_tensor=True)

    # 4. İki metin arasındaki kosinüs benzerliğini hesapla (0 ile 1 arasında bir skor)
    benzerlik_skoru = util.cos_sim(embedding_ogrenci, embedding_anahtar).item()

    # 5. Benzerlik skorunu 100'lük bir puanlama sistemine dönüştür
    puan_onerisi = benzerlik_skoru * 100

    print("\n--- OTOMATİK PUANLAMA SONUCU ---")
    print(f"Cevap Anahtarı ile Benzerlik Oranı: {benzerlik_skoru:.4f}")
    print(f"Önerilen Puan: {puan_onerisi:.2f} / 100")
    print("-" * 40)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]


--- OTOMATİK PUANLAMA SONUCU ---
Cevap Anahtarı ile Benzerlik Oranı: 0.6678
Önerilen Puan: 66.78 / 100
----------------------------------------


In [10]:
# Gerekli kütüphaneleri yükle
!pip install transformers accelerate

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Sadece puanlama yapıldıysa bu adıma geç
if 'puan_onerisi' in locals():
    # 1. Türkçe metin üretebilen bir model seçelim.
    # Not: Daha büyük modeller daha iyi sonuçlar verir ancak Colab'de kaynak kısıtlamaları olabilir.
    model_name_generative = "ytu-ce-cosmos/turkish-gpt2-large"
    tokenizer_generative = AutoTokenizer.from_pretrained(model_name_generative)
    model_generative = AutoModelForCausalLM.from_pretrained(model_name_generative)

    # Modelin "pad_token"u yoksa "eos_token"ı kullanmasını sağla
    if tokenizer_generative.pad_token is None:
        tokenizer_generative.pad_token = tokenizer_generative.eos_token

    # 2. Modele "Instruction Tuning" formatında bir girdi hazırla [cite: 83]
    prompt_template = f"""
    GÖREV: Bir öğretmensin. Aşağıdaki öğrenci cevabını, cevap anahtarına göre değerlendirip neden {puan_onerisi:.0f} puan aldığını açıkla.

    CEVAP ANAHTARI: {cevap_anahtari}

    ÖĞRENCİ CEVABI: {ogrenci_cevabi_tamami}

    DEĞERLENDİRME VE GERİ BİLDİRİM:
    """

    # 3. Girdiyi modelin anlayacağı formata dönüştür
    inputs_generative = tokenizer_generative(prompt_template, return_tensors="pt", max_length=512, truncation=True)

    # 4. Geri bildirim metnini üret
    output_ids = model_generative.generate(
        inputs_generative.input_ids,
        max_new_tokens=150,  # Üretilecek metnin uzunluğu
        num_beams=5,
        no_repeat_ngram_size=2,
        early_stopping=True
    )

    # 5. Üretilen metni decode et ve göster
    geri_bildirim = tokenizer_generative.decode(output_ids[0], skip_special_tokens=True)

    # Sadece üretilen kısmı almak için prompt'u metinden çıkar
    uretilen_kisim = geri_bildirim.replace(prompt_template, "").strip()

    print("\n--- OTOMATİK GERİ BİLDİRİM ---")
    print(uretilen_kisim)
    print("-" * 40)

tokenizer_config.json:   0%|          | 0.00/537 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/894 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.10G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



--- OTOMATİK GERİ BİLDİRİM ---
GÖREV: Bir öğretmensin. Aşağıdaki öğrenci cevabını, cevap anahtarına göre değerlendirip neden 67 puan aldığını açıkla.

    CEVAP ANAHTARI: 
    Prokaryot hücreler, zarla çevrili bir çekirdek ve organellere sahip değildir.
    Genetik materyal sitoplazmada serbest halde bulunur ve ribozom dışında belirgin bir organeli yoktur.
    Bu hücre tipi genellikle bakteriler ve arkelerde görülür.
    

    ÖĞRENCİ CEVABI: Questions
1) In $\mathbb{R}_2$, define $(x_1y) \oplus (x',y') = (x+x', y+y')$ and $c \odot (x,y) = (0,0)$. Is $\mathbb{R}_2$ a vector space with $\oplus$ and $\odot$? (c $\in \mathbb{R}$)
2) Let V be the set of all positive real numbers. Define $u \oplus v = u \cdot v$, $c \odot v = vc$ for any u, v $\in V$ and $c \in \mathbb{R}$. Prove that V is a vector space.
3) Determine whether or not the following set W is a subspace of V.
$$
a) W = \{ (2t, -3t) \mid t \in \mathbb{R} \}, V = \mathbb{R}_2.
$$
$$
b) W = \{ (2t+3, -4t) \mid t \in \mathbb{R} \}

In [11]:
import json
import re
from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# ==============================================================================
# PART 0: YOUR EXISTING OCR CODE (Assumed to be run before this)
# ... your code for running DotsOCR ...
# Let's assume 'output_text' is the variable holding the JSON string result.
# For demonstration, I'll create a sample 'output_text'.
# IN YOUR REAL CODE, USE THE OUTPUT FROM YOUR MODEL, NOT THIS SAMPLE.

sample_output_json = """
[
  {
    "bbox": [100, 120, 800, 160],
    "category": "Text",
    "text": "Soru 1: Prokaryot hücrelerin temel özelliklerini açıklayınız."
  },
  {
    "bbox": [110, 180, 790, 240],
    "category": "Text",
    "text": "Prokaryot hücrelerde çekirdek ve zarlı organel bulunmaz. Kalıtım materyali sitoplazmadadır."
  },
  {
    "bbox": [115, 250, 780, 290],
    "category": "Text",
    "text": "Ayrıca ribozomları vardır."
  },
  {
    "bbox": [100, 350, 800, 390],
    "category": "Text",
    "text": "2) Mitoz bölünmenin evreleri nelerdir? Sırasıyla yazınız."
  },
  {
    "bbox": [110, 410, 790, 450],
    "category": "Text",
    "text": "Mitoz evreleri profaz, metafaz, anafaz ve telofazdır."
  }
]
"""
# This simulates the output from your OCR model
output_text = [f"```json\n{sample_output_json}\n```"]

# ==============================================================================
# ADIM 1 (GÜNCELLENMİŞ): OCR ÇIKTISINDAN SORU VE CEVAPLARI AYIKLAMA
# ==============================================================================

print("--- 1. OCR Çıktısı İşleniyor ve Yapılandırılıyor ---")

qa_pairs = []
try:
    # Clean the JSON string from markdown code blocks
    clean_json_string = output_text[0].replace("```json\n", "").replace("\n```", "")
    layout_data = json.loads(clean_json_string)

    current_question = None
    current_answer_parts = []

    # Define a regex pattern to identify questions
    # This looks for lines starting with a number, or "Soru", "Problem" etc.
    question_pattern = re.compile(r'^\s*(\d+[\.\)])|^(Soru|Problem)\s*\d+', re.IGNORECASE)

    for element in layout_data:
        text = element.get('text', '').strip()
        if not text:
            continue

        # Check if the text matches our question pattern
        if question_pattern.match(text):
            # If we were already processing a question, save the previous one first
            if current_question:
                qa_pairs.append({
                    "question": current_question,
                    "answer": "\n".join(current_answer_parts)
                })

            # Start a new question
            current_question = text
            current_answer_parts = []
        elif current_question:
            # If we are in a question block, this text is part of the answer
            current_answer_parts.append(text)

    # Don't forget to add the last processed Q&A pair after the loop ends
    if current_question:
        qa_pairs.append({
            "question": current_question,
            "answer": "\n".join(current_answer_parts)
        })

    print(f"Başarıyla {len(qa_pairs)} adet Soru-Cevap bloğu bulundu.\n")
    for i, pair in enumerate(qa_pairs):
        print(f"--- BLOK {i+1} ---")
        print(f"Tespit Edilen Soru: {pair['question']}")
        print(f"Tespit Edilen Cevap: {pair['answer']}\n")

except (json.JSONDecodeError, IndexError) as e:
    print(f"Hata: OCR çıktısı işlenemedi. Çıktı geçerli formatta olmayabilir. Hata: {e}")

# ==============================================================================
# ADIM 2: HER SORU-CEVAP BLOĞUNU AYRI AYRI DEĞERLENDİRME
# ==============================================================================

if qa_pairs:
    print("\n--- 2. LLM ile Değerlendirme Başlatılıyor ---")

    # This should come from your system/database. It maps a question to its correct answer.
    # The keys should ideally match the question text for easy lookup.
    cevap_anahtarlari = {
        "Soru 1: Prokaryot hücrelerin temel özelliklerini açıklayınız.": "Prokaryot hücrelerde zarla çevrili çekirdek ve zarlı organeller bulunmaz. Genetik materyalleri (DNA) sitoplazma içinde serbest halde bulunur. Ribozom organeli vardır.",
        "2) Mitoz bölünmenin evreleri nelerdir? Sırasıyla yazınız.": "Mitoz bölünmenin evreleri sırasıyla Profaz, Metafaz, Anafaz ve Telofaz'dır. Bu evreleri sitokinez takip eder."
    }

    # Load the semantic similarity model once
    semantic_model = SentenceTransformer('dbmdz/bert-base-turkish-cased')

    # Loop through each extracted question-answer pair and evaluate it
    for i, pair in enumerate(qa_pairs):
        ogrenci_soru = pair['question']
        ogrenci_cevap = pair['answer']

        print(f"\n--- Soru {i+1} Değerlendiriliyor: '{ogrenci_soru[:50]}...' ---")

        # Find the correct answer key for the current question
        cevap_anahtari = cevap_anahtarlari.get(ogrenci_soru)

        if not cevap_anahtari:
            print(">> Uyarı: Bu soru için cevap anahtarı bulunamadı. Değerlendirme atlanıyor.")
            continue

        if not ogrenci_cevap.strip():
            print(">> Uyarı: Öğrenci bu soruya cevap vermemiş. Puan: 0")
            continue

        # --- Semantic Similarity and Scoring ---
        embedding_ogrenci = semantic_model.encode(ogrenci_cevap, convert_to_tensor=True)
        embedding_anahtar = semantic_model.encode(cevap_anahtari, convert_to_tensor=True)

        benzerlik_skoru = util.cos_sim(embedding_ogrenci, embedding_anahtar).item()
        puan_onerisi = benzerlik_skoru * 100

        print(f"Önerilen Puan: {puan_onerisi:.2f} / 100")

        # --- (Optional) Generate Feedback with a Generative LLM ---
        # This part can be computationally intensive to run for every answer.
        # You can enable it as needed.
        # ... (Code from the previous response for generating feedback) ...

--- 1. OCR Çıktısı İşleniyor ve Yapılandırılıyor ---
Başarıyla 2 adet Soru-Cevap bloğu bulundu.

--- BLOK 1 ---
Tespit Edilen Soru: Soru 1: Prokaryot hücrelerin temel özelliklerini açıklayınız.
Tespit Edilen Cevap: Prokaryot hücrelerde çekirdek ve zarlı organel bulunmaz. Kalıtım materyali sitoplazmadadır.
Ayrıca ribozomları vardır.

--- BLOK 2 ---
Tespit Edilen Soru: 2) Mitoz bölünmenin evreleri nelerdir? Sırasıyla yazınız.
Tespit Edilen Cevap: Mitoz evreleri profaz, metafaz, anafaz ve telofazdır.


--- 2. LLM ile Değerlendirme Başlatılıyor ---

--- Soru 1 Değerlendiriliyor: 'Soru 1: Prokaryot hücrelerin temel özelliklerini a...' ---
Önerilen Puan: 97.54 / 100

--- Soru 2 Değerlendiriliyor: '2) Mitoz bölünmenin evreleri nelerdir? Sırasıyla y...' ---
Önerilen Puan: 94.05 / 100


In [22]:
import re

questions_answers = []
current_question = None
current_answer = []
current_number = None

for element in layout_data:
    if 'text' in element and element['text']:
        line = element['text'].strip()

        match = re.match(r'^(\d+)[\).]', line)  # detect question number
        if match:
            # Save old one
            if current_question and current_answer:
                questions_answers.append({
                    "number": current_number,
                    "question": current_question.strip(),
                    "answer": "\n".join(current_answer).strip()
                })
            # Start new one
            current_number = int(match.group(1))
            current_question = line
            current_answer = []
        else:
            current_answer.append(line)

# Save last
if current_question and current_answer:
    questions_answers.append({
        "number": current_number,
        "question": current_question.strip(),
        "answer": "\n".join(current_answer).strip()
    })


print("\n--- DETECTED QUESTIONS & ANSWERS ---")
for i, qa in enumerate(questions_answers, 1):
    print(f"Q{i}: {qa['question']}")
    print(f"A{i}: {qa['answer']}\n")



--- DETECTED QUESTIONS & ANSWERS ---
Q1: 2) Mitoz bölünmenin evreleri nelerdir? Sırasıyla yazınız.
A1: Mitoz evreleri profaz, metafaz, anafaz ve telofazdır.



In [21]:
results = []
# Create a mapping from question text to question number for easy lookup
question_to_number = {qa["question"]: qa["number"] for qa in questions_answers}

for qa in questions_answers:
    student_answer = qa["answer"]
    # Find the correct answer key by matching the question text
    correct_answer = None
    for key, value in cevap_anahtarlari.items():
        if key in qa["question"]: # Simple check if the question text is in the key
             correct_answer = value
             break

    if not correct_answer:
        print(f"Uyarı: Soru '{qa['question'][:50]}...' için cevap anahtarı bulunamadı. Puanlama atlanıyor.")
        continue

    emb_student = model_semantic.encode(student_answer, convert_to_tensor=True)
    emb_key = model_semantic.encode(correct_answer, convert_to_tensor=True)

    score = util.cos_sim(emb_student, emb_key).item() * 100
    results.append({
        "number": qa["number"],
        "question": qa["question"],
        "student_answer": student_answer,
        "score": score
    })

# Print results
for r in results:
    print(f"Soru {r['number']}: {r['question']}")
    print(f"Puan: {r['score']:.2f}/100\n")

Soru 2: 2) Mitoz bölünmenin evreleri nelerdir? Sırasıyla yazınız.
Puan: 94.05/100



In [26]:
import json
import re
from sentence_transformers import SentenceTransformer, util
from transformers import AutoModelForCausalLM, AutoProcessor
from qwen_vl_utils import process_vision_info

# ---- Load DotsOCR model ----
model_path = "./weights/DotsOCR"
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    attn_implementation="flash_attention_2",
    torch_dtype="auto",
    device_map="auto",
    trust_remote_code=True
)
processor = AutoProcessor.from_pretrained(model_path, trust_remote_code=True)

# ---- Image path ----
image_path = "/content/sinav2.png"  # update with your uploaded path

# ---- Prompt for OCR ----
prompt = """
Please output the layout information from the exam image, including each layout element's bbox, category, and text.

Rules:
- Output must be a single JSON array of objects.
- Keep original text, do not translate.
- Preserve reading order.
"""

messages = [
    {"role": "user", "content": [{"type": "image", "image": image_path}, {"type": "text", "text": prompt}]}
]

# Prepare input for model
text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(text=[text], images=image_inputs, videos=video_inputs, padding=True, return_tensors="pt").to("cuda")

# OCR inference
generated_ids = model.generate(**inputs, max_new_tokens=8000)
generated_ids_trimmed = [out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)]
output_text = processor.batch_decode(generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False)

# ---- Parse JSON safely ----
try:
    clean_json_string = output_text[0].replace("```json\n", "").replace("\n```", "")
    layout_data = json.loads(clean_json_string)
except json.JSONDecodeError:
    print("OCR output is not valid JSON")
    layout_data = []

# ---- Split into questions & answers ----
questions_answers = []
current_question = None
current_answer = []
current_number = None

for element in layout_data:
    if 'text' in element and element['text']:
        line = element['text'].strip()

        # Detect start of new question (e.g., "1)")
        match = re.match(r'^(\d+)[\).]', line)
        if match:
            # Save old Q/A
            if current_question and current_answer:
                questions_answers.append({
                    "number": current_number,
                    "question": current_question.strip(),
                    "answer": "\n".join(current_answer).strip()
                })
            # New question
            current_number = int(match.group(1))
            current_question = line
            current_answer = []
        elif line.lower().startswith("a. cevap"):
            # It's the answer line
            answer_text = line.split("cevap", 1)[-1].replace(":", "").strip()
            current_answer.append(answer_text)
        else:
            # Continue question text
            if current_question and not current_answer:
                current_question += " " + line
            elif current_answer:
                current_answer.append(line)

# Save last one
if current_question and current_answer:
    questions_answers.append({
        "number": current_number,
        "question": current_question.strip(),
        "answer": "\n".join(current_answer).strip()
    })

print("\n--- DETECTED Q/A ---")
for qa in questions_answers:
    print(f"{qa['number']}) {qa['question']}")
    print(f"Answer: {qa['answer']}\n")

# ---- Answer key for grading ----
answer_key_dict = {
    1: "O(N^2)",
    2: "O(N*log(N))"
}

# ---- Load semantic similarity model ----
model_semantic = SentenceTransformer('dbmdz/bert-base-turkish-cased')

# ---- Grade each answer ----
results = []
for qa in questions_answers:
    student_answer = qa["answer"]
    correct_answer = answer_key_dict.get(qa["number"], "")

    emb_student = model_semantic.encode(student_answer, convert_to_tensor=True)
    emb_key = model_semantic.encode(correct_answer, convert_to_tensor=True)

    score = util.cos_sim(emb_student, emb_key).item() * 100
    results.append({
        "number": qa["number"],
        "question": qa["question"],
        "student_answer": student_answer,
        "score": score
    })

# ---- Show results ----
total_score = sum(r["score"] for r in results) / len(results)
print("\n--- GRADING RESULTS ---")
for r in results:
    print(f"Soru {r['number']}: {r['score']:.2f}/100")
print(f"\nOverall average: {total_score:.2f}/100")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.



--- DETECTED Q/A ---


ZeroDivisionError: division by zero

In [33]:
import json, re
from typing import List, Dict, Any
from sentence_transformers import SentenceTransformer, util

# ============ 1) Robust OCR output parser ============

def coerce_layout(output_text) -> List[Dict[str, Any]]:
    """
    Accepts:
      - raw string
      - list[str] (e.g. ['[...]'])
      - list[dict] (already parsed)
      - code-fenced variations
    Returns: list of dicts with keys like 'bbox', 'category', 'text'
    """
    def _json_try(s: str):
        s = s.strip()
        # strip code fences ```json ... ```
        s = re.sub(r"^```(?:json)?\s*", "", s)
        s = re.sub(r"\s*```$", "", s)
        # If the model returned extra text, try to extract the first JSON array
        m = re.search(r"\[\s*{.*}\s*\]", s, flags=re.DOTALL)
        if m:
            s = m.group(0)
        return json.loads(s)

    # Already a list of dicts?
    if isinstance(output_text, list) and output_text and isinstance(output_text[0], dict):
        return output_text

    # List with a single string?
    if isinstance(output_text, list) and len(output_text) == 1 and isinstance(output_text[0], str):
        try:
            return _json_try(output_text[0])
        except json.JSONDecodeError:
            pass  # fall through

    # Bare string?
    if isinstance(output_text, str):
        try:
            return _json_try(output_text)
        except json.JSONDecodeError:
            pass

    # Last resort: try json on each element if it's a list of strings
    if isinstance(output_text, list) and all(isinstance(x, str) for x in output_text):
        for s in output_text:
            try:
                return _json_try(s)
            except json.JSONDecodeError:
                continue

    # Could not parse → return empty
    return []

# ============ 2) Q/A extractor (supports subparts like (a), (b)) ============

PART_RE = re.compile(r"^\(?([a-z])\)?(\.|$|\s)", re.IGNORECASE)
QUESTION_START_RE = re.compile(r"(?:^|\s)Soru\s*(\d+)\b", re.IGNORECASE)
NUMBER_HEAD_RE = re.compile(r"^(\d+)[\).]")  # e.g. "1)" or "1."

def normalize_text(s: str) -> str:
    return re.sub(r"\s+", " ", s).strip()

def extract_qas(layout_data: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """
    Converts layout elements into a list of:
      { qid: "1a", number: 1, part: "a", question: "...", answer: "..." }
    Handles patterns like:
      - "Soru 1. ..."
      - "(a) (8 puan)" / "a." / "a)"
      - "Cevap:" / "Büyüme Düzeni:"
    """
    if not layout_data:
        return []

    # Sort by reading order just in case
    def key_bbox(e):
        b = e.get("bbox", [0, 0, 0, 0])
        return (b[1], b[0])  # y1, x1

    layout_sorted = sorted(layout_data, key=key_bbox)

    qas = []
    curr_qnum = None
    curr_part = None
    curr_qtext_parts = []
    curr_ans_parts = []
    have_answer_started = False

    def flush():
        nonlocal curr_qnum, curr_part, curr_qtext_parts, curr_ans_parts, have_answer_started
        if curr_qnum is None or curr_part is None:
            # nothing to flush
            curr_qtext_parts, curr_ans_parts = [], []
            have_answer_started = False
            return
        qid = f"{curr_qnum}{curr_part}"
        qtext = normalize_text("\n".join(curr_qtext_parts))
        ans = normalize_text("\n".join(curr_ans_parts))
        if qtext or ans:
            qas.append({
                "qid": qid,
                "number": curr_qnum,
                "part": curr_part,
                "question": qtext,
                "answer": ans
            })
        # reset for next part
        curr_part = None
        curr_qtext_parts = []
        curr_ans_parts = []
        have_answer_started = False

    for el in layout_sorted:
        txt = el.get("text", "")
        if not txt:
            continue
        line = txt.strip()

        # Detect "Soru X"
        m_q = QUESTION_START_RE.search(line)
        if m_q:
            # New question block → flush any previous part
            flush()
            curr_qnum = int(m_q.group(1))
            curr_part = None
            curr_qtext_parts = [line]
            curr_ans_parts = []
            have_answer_started = False
            continue

        # Detect explicit numeric start "1." if "Soru" wasn't present
        if curr_qnum is None:
            m_head = NUMBER_HEAD_RE.match(line)
            if m_head:
                flush()
                curr_qnum = int(m_head.group(1))
                curr_part = None
                curr_qtext_parts = [line]
                curr_ans_parts = []
                have_answer_started = False
                continue

        # Detect subpart markers like "(a)", "a)", "a."
        m_p = PART_RE.match(line)
        if m_p:
            # Starting a new part → flush previous part
            if curr_part is not None:
                flush()
            curr_part = m_p.group(1).lower()
            curr_qtext_parts = [line]
            curr_ans_parts = []
            have_answer_started = False
            continue

        # If a question is active but no part yet, keep appending to the question text
        # until we see a part marker.
        if curr_qnum is not None and curr_part is None:
            curr_qtext_parts.append(line)
            continue

        # Answer triggers
        is_answer_trigger = (
            re.search(r"\bcevap\b", line, re.IGNORECASE) or
            re.search(r"büyüme\s*düzeni", line, re.IGNORECASE) or
            re.search(r"\byanıt\b", line, re.IGNORECASE)
        )

        if curr_qnum is not None and curr_part is not None:
            if is_answer_trigger:
                have_answer_started = True
                # Normalize common prefixes
                line_clean = re.sub(r"(?i)^(a\.\s*)?cevap\s*:?\s*", "", line)
                line_clean = re.sub(r"(?i)^büyüme\s*düzeni\s*:?\s*", "", line_clean)
                curr_ans_parts.append(line_clean.strip())
            else:
                # Before answer starts → keep building the subpart question
                if not have_answer_started:
                    curr_qtext_parts.append(line)
                else:
                    # After answer started → multi-line answer continuation
                    curr_ans_parts.append(line)

    # Flush last part
    if curr_qnum is not None and curr_part is not None:
        flush()

    return qas

# ============ 3) Grading (safe against empty results) ============

def grade_qas(qas: List[Dict[str, Any]], answer_key_dict: Dict[str, str]):
    """
    qas: output of extract_qas()
    answer_key_dict: keys like "1a", "1b"
    Returns: (results_list, overall_average)
    """
    # Pick a multilingual ST model to be robust in TR/EN
    st_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

    results = []
    for qa in qas:
        qid = qa["qid"]
        student_answer = qa.get("answer", "")
        key_answer = answer_key_dict.get(qid, "")
        if not student_answer or not key_answer:
            score = 0.0
        else:
            emb_s = st_model.encode(student_answer, convert_to_tensor=True)
            emb_k = st_model.encode(key_answer, convert_to_tensor=True)
            score = float(util.cos_sim(emb_s, emb_k).item() * 100.0)

        results.append({
            "qid": qid,
            "question": qa.get("question", ""),
            "student_answer": student_answer,
            "key_answer": key_answer,
            "score": score
        })

    overall = (sum(r["score"] for r in results) / len(results)) if results else 0.0
    return results, overall

# ============ 4) Example usage with your sample OCR output ============

# Example "bad" shape from your message:
ocr_output_example = ['[{"bbox": [10, 3, 377, 22], "category": "Section-header", "text": "Soru 1. (15 puan) Algoritm a Analizi (Analysis of Algorithms)"}, {"bbox": [10, 37, 568, 74], "category": "Text", "text": "Aşağıdaki verilen kod parçaları için çalışma sürelerinin büyüme düzenini (order of growth)\\nN\'nin bir fonksiyonu cinsinden --notasyonu ile belirtiniz."}, {"bbox": [10, 92, 90, 109], "category": "Section-header", "text": "(a) (8 puan)"}, {"bbox": [10, 127, 261, 194], "category": "Text", "text": "int m = 0;\\nfor (int i = N; i >0; i--)\\n    for (int j = 0; j < i; j++)\\n        m++;"}, {"bbox": [10, 194, 276, 257], "category": "Text", "text": "Büyüme Düzeni: ~ O(N²)"}, {"bbox": [10, 472, 90, 489], "category": "Section-header", "text": "(b) (7 puan)"}, {"bbox": [10, 506, 292, 574], "category": "Text", "text": "int k = 0;\\nfor (int i = 1; i <= N; i++)\\n    for (int j = 1; j <= N; j++)\\n        k++;"}, {"bbox": [10, 570, 363, 644], "category": "Text", "text": "Büyüme Düzeni: ~ O(n · log(n))"}]']

layout = coerce_layout(ocr_output_example)
qas = extract_qas(layout)

# Fill your answer key here (keys like "1a", "1b")
# (These are examples; replace with your ground truth.)
answer_key_dict = {
    "1a": " O(N²)",
    "1b": " O(N log(N))"
}

results, overall = grade_qas(qas, answer_key_dict)

print("--- PARSED Q/A ---")
for qa in qas:
    print(f"{qa['qid']}:")
    print("Q:", qa['question'])
    print("A:", qa['answer'])
    print()

print("--- GRADES ---")
for r in results:
    print(f"{r['qid']}: {r['score']:.2f}/100")
print(f"Overall: {overall:.2f}/100")

# if acc is higher than 75%, that means that question is correct




--- PARSED Q/A ---
1a:
Q: (a) (8 puan) int m = 0; for (int i = N; i >0; i--) for (int j = 0; j < i; j++) m++;
A: ~ O(N²)

1b:
Q: (b) (7 puan) int k = 0; for (int i = 1; i <= N; i++) for (int j = 1; j <= N; j++) k++;
A: ~ O(n · log(n))

--- GRADES ---
1a: 90.78/100
1b: 90.90/100
Overall: 90.84/100


In [34]:
correct_count = 0
total_questions = len(results)

for r in results:
    if r['score'] > 75:
        correct_count += 1

print(f"Based on a score threshold of 75/100:")
print(f"{correct_count} out of {total_questions} questions are considered correct.")

Based on a score threshold of 75/100:
2 out of 2 questions are considered correct.
